<a href="https://colab.research.google.com/github/prathamnishad/experimental/blob/master/intern2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import zipfile
%matplotlib inline
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import os
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
import time
from sklearn.metrics import classification_report, accuracy_score
from sklearn import preprocessing



local_zip = '/content/drive/My Drive/raw.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/data')
raw=os.path.join("/tmp/data/raw")
train_gyro_phone=os.path.join("/tmp/data/raw/train/phone/gyro")
print('total train file:', len(os.listdir(train_gyro_phone)))
train_accel_phone=os.path.join("/tmp/data/raw/train/phone/accel")
print('total train file:', len(os.listdir(train_accel_phone)))
train_gyro_watch=os.path.join("/tmp/data/raw/train/watch/gyro")
print('total train file:', len(os.listdir(train_gyro_watch)))
train_accel_watch=os.path.join("/tmp/data/raw/train/watch/accel")
print('total train file:', len(os.listdir(train_accel_watch)))

test_gyro_phone=os.path.join("/tmp/data/raw/test/phone/gyro")
print('total test file:', len(os.listdir(test_gyro_phone)))
test_accel_phone=os.path.join("/tmp/data/raw/test/phone/accel")
print('total test file:', len(os.listdir(test_accel_phone)))
test_gyro_watch=os.path.join("/tmp/data/raw/test/watch/gyro")
print('total test file:', len(os.listdir(test_gyro_watch)))
test_accel_watch=os.path.join("/tmp/data/raw/test/watch/accel")
print('total test file:', len(os.listdir(test_accel_watch)))


total train file: 20
total train file: 20
total train file: 20
total train file: 20
total test file: 14
total test file: 14
total test file: 14
total test file: 14


In [2]:
#for train
df=pd.DataFrame()
for n in os.listdir(train_gyro_phone):
    a=pd.read_csv(os.path.join(train_gyro_phone,n),sep=",",header=None)
    df=pd.concat([df,a])
for n in os.listdir(train_accel_phone):
    a=pd.read_csv(os.path.join(train_accel_phone,n),sep=",",header=None)
    df=pd.concat([df,a])
  
for n in os.listdir(train_gyro_watch):
    a=pd.read_csv(os.path.join(train_gyro_watch,n),sep=",",header=None)
    df=pd.concat([df,a])

for n in os.listdir(train_accel_watch):
    a=pd.read_csv(os.path.join(train_accel_watch,n),sep=",",header=None)
    df=pd.concat([df,a])
d_train=df
df.head()

,0,1,2,3,4,5
0,1612,A,3952189047449,0.817322,-5.137268,-0.20063782;
1,1612,A,3952239401453,-1.029923,-3.979431,-1.3396301;
2,1612,A,3952289755457,0.493454,-1.189850,-2.666687;
3,1612,A,3952340109460,2.300003,-2.157044,-1.5438995;
4,1612,A,3952390463464,1.797806,-2.138672,1.0609283;


In [3]:
#for test
df_test=pd.DataFrame()
for n in os.listdir(test_gyro_phone):
    a=pd.read_csv(os.path.join(test_gyro_phone,n),sep=",",header=None)
    df_test=pd.concat([df_test,a])
for n in os.listdir(test_accel_phone):
    a=pd.read_csv(os.path.join(test_accel_phone,n),sep=",",header=None)
    df_test=pd.concat([df_test,a])
  
for n in os.listdir(test_gyro_watch):
    a=pd.read_csv(os.path.join(test_gyro_watch,n),sep=",",header=None)
    df_test=pd.concat([df_test,a])

for n in os.listdir(test_accel_watch):
    a=pd.read_csv(os.path.join(test_accel_watch,n),sep=",",header=None)
    df_test=pd.concat([df_test,a])
d_test=df_test
df_test.head()

,0,1,2,3,4,5
0,1629,A,1127073874136682,-0.664368,-0.642181,-0.3517151;
1,1629,A,1127073924490686,-0.154068,-0.731796,-0.33454895;
2,1629,A,1127073974844690,0.290558,-0.411285,-0.12980652;
3,1629,A,1127074025198693,0.679916,-0.354401,-0.27474976;
4,1629,A,1127074075552697,0.757401,-0.079544,-0.45448303;


In [0]:
#function to preprocess data
def make_df(df):
  df.columns=["id","activity","time","x","y","z"]#column names
  df['z'] = df['z'].apply(lambda x: float(x[:-1]))#converting the z value column to float
  df['time'] = df['time'].apply(lambda x: x//1000000)#converting timestamp in micro seconds to datetime objects
  df['time'] = df['time'].apply(lambda x: datetime.fromtimestamp(x))
  for n in ("month","week","day","dayofweek","dayofyear","is_month_end","is_month_start","is_quarter_end","is_quarter_start","is_year_end","is_year_start"):
      df[n]=getattr(df.time.dt,n)# extracting extra data from time column
  df.drop(columns="time",inplace=True)

  for n in ["is_month_end","is_month_start","is_quarter_end","is_quarter_start","is_year_end","is_year_start"]:
    df[n]=df[n].apply(lambda x:int(x))#converting to int from boolean i don't know why(o_o)

  act=["A","B","C","D","E","F","G","H","I","J","K","L","M","O","P","Q","R","S"]#converting activity column from categorical data to numerical data
  df["activity"]=df["activity"].apply(lambda a: act.index(a))
  df = df.sample(frac=1).reset_index(drop=True)
  df.drop(columns="id",inplace=True)#dropping id column as it is of no use
  return df
  

In [0]:
#for train
df_train=make_df(df)
#for test
df_test=make_df(df_test)

In [0]:
dfa=pd.DataFrame()#single data frame for whole data train + test. Later divided into train and test set 
dfa=pd.concat([dfa,df_test])
dfa=pd.concat([dfa,df_train])

In [0]:
train_y=dfa["activity"]
train_x=dfa.drop(columns="activity")
x_train, x_test, y_train, y_test = train_test_split(train_x,train_y , train_size = 0.9, random_state =  90)
train=len(x_train)
test=len(x_test)
mm_scaler = preprocessing.Normalizer().fit(x_train)

In [0]:
x_train=mm_scaler.transform(x_train)
x_test=mm_scaler.transform(x_test)


In [9]:
model_tree = tree.DecisionTreeClassifier()

start = time.time()
model_tree.fit(x_train, y_train)
end = time.time()
print(end - start)

y_pred= model_tree.predict(x_test)
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy of {model_tree} is {acc}")


263.41410994529724
Accuracy of DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best') is 0.9116025748499952


In [11]:
Random_classifier = RandomForestClassifier(n_estimators = 50)
start = time.time()
Random_classifier.fit(x_train, y_train)
end = time.time()
print(end - start)
y_pred = Random_classifier.predict(x_test)
result = accuracy_score(y_test,y_pred)
print("Accuracy:",result)

1692.355530500412
Accuracy: 0.9357933290622369


In [0]:
#Decision tree takes much less time to train in ralation to Random Forest.
#Accuracy of Random Forest is asways more than that of Decision Tree 

In [0]:
activity_encoder=["A","B","C","D","E","F","G","H","I","J","K","L","M","O","P","Q","R","S"]

In [0]:
#Decision Tree evaluation
eval_data=make_df(eval_data)#eval_data is the evaluation dataframe
eval_data=mm.scaler.transform(eval_data)#provided by you
y_pred= model_tree.predict(eval_data)
actual_pred=[]
for n in y_pred:
  actual_pred.append(activity_encoder[n])

In [0]:
#Random Forest evaluation
eval_data=make_df(eval_data)#eval_data is the evaluation dataframe
eval_data=mm.scaler.transform(eval_data)#provided by you
y_pred= Random_classifier.predict(eval_data)
actual_pred=[]
for n in y_pred:
  actual_pred.append(activity_encoder[n])